# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [0]:
import pandas as pd
import numpy as np
import folium


In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 586 (delta 5), reused 8 (delta 2), pack-reused 572
Receiving objects: 100% (586/586), 130.66 MiB | 24.35 MiB/s, done.
Resolving deltas: 100% (237/237), done.
Checking out files: 100% (219/219), done.


In [0]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv(path+'/DATA/train_cleaned.csv')

In [0]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [0]:
from sklearn.cluster import KMeans

In [0]:
#define number of clusters and create instance
clusters=100
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [7]:
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [0]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [0]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [11]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [0]:
def show_cluster(cluster_number, predicted_coordinates):
  cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)

  coordinates_of_cluster = coordinates[predicted_coordinates == cluster_number]
  for i in range(coordinates_of_cluster.shape[0]):
    row = coordinates_of_cluster.iloc[i]
    folium.CircleMarker([row[0], row[1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                      ).add_to(cluster_map)
    folium.CircleMarker([row[2], row[3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                      ).add_to(cluster_map)
                      
  folium.CircleMarker([centers[cluster_number,0], centers[cluster_number,1]], radius=3,                
                      color="black", 
                      fill_opacity=0.9
                    ).add_to(cluster_map)
  folium.CircleMarker([centers[cluster_number,2], centers[cluster_number,3]], radius=3,                
                      color="black", 
                      fill_opacity=0.9
                    ).add_to(cluster_map)
  folium.PolyLine([ [centers[cluster_number,0],centers[cluster_number,1]] , [centers[cluster_number,2],centers[cluster_number,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)
  return cluster_map

In [73]:
predicted_coordinates = myKMeans.predict(coordinates)
show_cluster(0, predicted_coordinates)

## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.

In [0]:
def cluster_var(cluster_number):
  coordinates_of_cluster = coordinates[predicted_coordinates == cluster_number]
  return coordinates_of_cluster.var()

In [89]:
cluster_var(99)

pickup_latitude      0.000609
pickup_longitude     0.000949
dropoff_latitude     0.000450
dropoff_longitude    0.001067
dtype: float64

In [90]:
cluster_var(9)

pickup_latitude      0.000055
pickup_longitude     0.000027
dropoff_latitude     0.000047
dropoff_longitude    0.000031
dtype: float64